## Ансамбли и полносвязные нейронные сети
В этом ноутбуке вам нужно обучить модели на датасете классификации из предыдущего ноутбука и сравнить результаты. Вам будет предоставлен baseline, на основе которого вы будете доделывать предсказывающие модели. Оценка лабы будет зависеть от ROC-AUC на тестовых данных по следующим критериям:
\
AUC - на тестовых данных
- $AUC \leq 0.76$ - 0 баллов
- $0.76 < AUC \leq 0.77$ - 2 балла
- $0.77 < AUC \leq 0.78$ - 4 балла
- $0.78 < AUC \leq 0.79$ - 6 баллов
- $0.79 < AUC \leq 0.80$ - 8 баллов
- $AUC > 0.80$ - 10 баллов


In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, precision_score, recall_score, roc_curve, accuracy_score

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('german.csv', sep=';') #читаем данные
print(data)

In [ ]:
plt.hist(y_train, bins=2, edgecolor='k') # визуализация тренинговых таргетов
plt.xticks([0, 1])
plt.xlabel('Class (0: Non-Creditworthy, 1: Creditworthy)')
plt.ylabel('Count')
plt.title('Distribution of Classes in Training Data')
plt.show()

In [49]:
data = pd.read_csv('german.csv', sep=';')

#X = data.iloc[:, 1:].to_numpy()
#"""
XX = data[
    [
    'Account_Balance', 
    'Duration_of_Credit_monthly', 
    'Payment_Status_of_Previous_Credit', 
    #'Purpose',
    #'Credit_Amount',
    #'Value_Savings_Stocks',
    'Length_of_current_employment', 
    #'Instalment_per_cent',
    'Sex_Marital_Status', 
    'Guarantors', #
    #'Duration_in_Current_address',
    'Most_valuable_available_asset', 
    #'Age_years',
    #'Concurrent_Credits',
    #'Type_of_apartment',
    'No_of_Credits_at_this_Bank', 
    #'Occupation',
    #'No_of_dependents',
    #'Telephone',
    #'Foreign_Worker'
    ]
] 
X = XX.to_numpy()
#"""
y = data.iloc[:, 0].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
data.head()

,Creditability,Account_Balance,Duration_of_Credit_monthly,Payment_Status_of_Previous_Credit,Purpose,Credit_Amount,Value_Savings_Stocks,Length_of_current_employment,Instalment_per_cent,Sex_Marital_Status,...,Duration_in_Current_address,Most_valuable_available_asset,Age_years,Concurrent_Credits,Type_of_apartment,No_of_Credits_at_this_Bank,Occupation,No_of_dependents,Telephone,Foreign_Worker
0,1,1,18,4,2,1049,1,2,4,2,...,4,2,21,3,1,1,3,1,1,1
1,1,1,9,4,0,2799,1,3,2,3,...,2,1,36,3,1,2,3,2,1,1
2,1,2,12,2,9,841,2,4,2,2,...,4,1,23,3,1,1,2,1,1,1
3,1,1,12,4,0,2122,1,3,3,3,...,2,1,39,3,1,2,2,2,1,2
4,1,1,12,4,0,2171,1,3,4,3,...,4,2,38,1,2,2,2,1,1,2


In [54]:
# Пример балансировки классов для RandomForestClassifier
rf_model = RandomForestClassifier(
    n_estimators=300, #300 этапы повышения
    criterion='log_loss', #entropy критерий классифицирования
    max_depth=40, #40 макимальная глубина
    min_samples_split=40, #45 минимальное кол-во разбиений на сэмплы
    min_samples_leaf=33, #33 миимальные количество выборок для узла
    class_weight='balanced',  #balanced  веса классов
    random_state=48, #48 рандомизация загрузки выборок
)
rf_model.fit(X_train, y_train) # обучение модели

# Прогноз на тестовых данных
rf_pred = rf_model.predict(X_test) # предсказание даннных

# Расчет метрик для Random Forest
rf_roc_auc = roc_auc_score(y_test, rf_pred)
rf_accuracy = accuracy_score(y_test, rf_pred)
rf_precision = precision_score(y_test, rf_pred)
rf_recall = recall_score(y_test, rf_pred)

print("Random Forest метрики:")
print(f"ROC AUC: {rf_roc_auc:.2f}") # метрика оценки качества модели
print(f"Accuracy: {rf_accuracy:.2f}") # точность модели
print(f"Precision: {rf_precision:.2f}") # показывает, как хорошо модель находит все позитивные сэмплы
print(f"Recall: {rf_recall:.2f}") # классификатор ошибок, отношение 2*precision*recall/(precision+recall)

#Градиентный бустинг – это продвинутый алгоритм машинного обучения для решения задач классификации и регрессии. 
#Он строит предсказание в виде ансамбля слабых предсказывающих моделей, которыми в основном являются деревья решений.
# Обучение Gradient Boosting
gb_model = GradientBoostingClassifier(    
    max_depth=40,
    criterion='friedman_mse',
    learning_rate=0.01, # 0.13 скорость обучения
    n_estimators=250, # 500   число итераций 
    random_state=48, # 48    рандомизация загрузки выборок
    min_samples_split=120, # 120 минимальное кол-во разбиений на сэмплы
    min_samples_leaf=100, # 100   миимальные количество выборок для узла
    ccp_alpha=0.000008, #000008           # Зафиксированный seed для воспроизводимости результатов
)
"""parameters = {
    "n_estimators":[5,50,250,500],
    "max_depth":[1,3,5,7,9],
    "learning_rate":[0.01,0.1,1,10,100]
}
cv = GridSearchCV(gb_model,parameters,cv=5)
cv.fit(X_train,y_train)"""
"""    max_depth=40,
    learning_rate=0.13, # 0.13 скорость обучения
    n_estimators=500, # 500   число итераций 
    random_state=42, # 48    рандомизация загрузки выборок
    min_samples_split=120, # 120 минимальное кол-во разбиений на сэмплы
    min_samples_leaf=100, # 100   миимальные количество выборок для узла
    ccp_alpha=0.000008, #000008"""
"""def display(results):
    print(f'Best parameters are: {results.best_params_}')
    print("\n")
    mean_score = results.cv_results_['mean_test_score']
    std_score = results.cv_results_['std_test_score']
    params = results.cv_results_['params']
    for mean,std,params in zip(mean_score,std_score,params):
        print(f'{round(mean,3)} + or -{round(std,3)} for the {params}')
display(cv)"""
# Обучение модели на обучающих данных
gb_model.fit(X_train, y_train)

# Прогноз вероятностей на тестовых данных
gb_pred = gb_model.predict_proba(X_test)[:, 1]

# Расчет метрик для Gradient Boosting
gb_roc_auc = roc_auc_score(y_test, gb_pred)

# Вывод метрик для оценки производительности модели
print("\nGradient Boosting метрики:")
print(f"ROC AUC: {gb_roc_auc:.2f}")
print(f"ROC AUC: {gb_roc_auc:.5f}")

Random Forest метрики:
ROC AUC: 0.80
Accuracy: 0.78
Precision: 0.91
Recall: 0.75

Gradient Boosting метрики:
ROC AUC: 0.79
ROC AUC: 0.78839


In [55]:
# Обучение MLP (Multi-Layer Perceptron) нейронной сети
# MPL - то модель искусственной нейронной сети с прямой связью, которая 
# сопоставляет входные наборы данных с набором соответствующих выходных данных
# Также используется для оптимизации функцииии логарифмических потерь
mlp_model = MLPClassifier(
    hidden_layer_sizes=(100,100), # 100, 100 кол-во нейронов в слое
    max_iter=500,  #500 максимальное кол-во классификаторов
    random_state=42, #42  рандомизация загрузки выборок
    solver='adam', #adam оптимизатор веса
    learning_rate='constant', # constant график скорости обучения для обновления веса
    alpha=0.000078, #000078 сила члена регуляризации, не очень понял этот параметр
    learning_rate_init=0.0015, #0015 начальная скорость обучения

) #0.77232
mlp_model.fit(X_train, y_train) # обучаем модель

# Прогноз на тестовых данных
mlp_pred = mlp_model.predict_proba(X_test)[:,1]

# Расчет метрик для MLP нейронной сети
mlp_roc_auc = roc_auc_score(y_test, mlp_pred)

print("\nMLP (Neural Network) метрики:")
print(f"ROC AUC: {mlp_roc_auc:.2f}") # метрика оценки качества модели
print(f"ROC AUC: {mlp_roc_auc:.5f}")


MLP (Neural Network) метрики:
ROC AUC: 0.81
ROC AUC: 0.80593


### Экспериментируйте
Для получения лучшего качества придется поэкспериментировать. Подсказка: попробуйте оптимизировать гиперпараметры модели